In [1]:
import yfinance as yf

aapl = yf.Ticker("AAPL")

# # get all stock info
# print(aapl.info['sharesOutstanding'] * aapl.info['previousClose'])

aapl.info
# get historical market data
# hist = aapl.history(period="1mo")

# show meta information about the history (requires history() to be called first)
# aapl.history_metadata

# # show actions (dividends, splits, capital gains)
# msft.actions
# msft.dividends
# msft.splits
# msft.capital_gains  # only for mutual funds & etfs

# # show share count
# msft.get_shares_full(start="2022-01-01", end=None)

# # show financials:
# # - income statement
# msft.income_stmt
# aapl.quarterly_income_stmt
# # - balance sheet
# msft.balance_sheet
# msft.quarterly_balance_sheet
# # - cash flow statement
# msft.cashflow
# msft.quarterly_cashflow
# # see `Ticker.get_income_stmt()` for more options

# # show holders
# msft.major_holders
# msft.institutional_holders
# msft.mutualfund_holders
# msft.insider_transactions
# msft.insider_purchases
# msft.insider_roster_holders

# # show recommendations
# msft.recommendations
# msft.recommendations_summary
# msft.upgrades_downgrades

# # Show future and historic earnings dates, returns at most next 4 quarters and last 8 quarters by default.
# # Note: If more are needed use msft.get_earnings_dates(limit=XX) with increased limit argument.
# msft.earnings_dates

# # show ISIN code - *experimental*
# # ISIN = International Securities Identification Number
# msft.isin

# # show options expirations
# msft.options

# # show news
# msft.news

# get option chain for specific expiration
# opt = msft.option_chain('2024-07-08')
# data available via: opt.calls, opt.puts

3346513977016.3203


In [29]:
import yfinance as yf

# Fetch the ticker data
ticker = yf.Ticker('AAPL')

# Specify the date for the income statement
income_statement_date = '2023-09-30'

# Step 1: Check the structure and available dates in the DataFrame
print("Quarterly Income Statement:")
print(ticker.quarterly_income_stmt)
print("\nAvailable Dates:")
print(ticker.quarterly_income_stmt.columns)

# Step 2: Access 'Basic Average Shares' for the specified date
try:
    total_shares = ticker.quarterly_income_stmt.loc['Basic Average Shares', income_statement_date]
    print(f"Total Shares on {income_statement_date}: {total_shares}")
except KeyError as e:
    print(f"Error accessing 'Basic Average Shares': {e}")
    total_shares = None

# Step 3: Fetch the stock price for the closest date to the specified income statement date
try:
    stock_price_data = ticker.history(start='2023-09-26', end='2023-09-30')  # Adjusting to ensure we include '2023-09-30'
    if not stock_price_data.empty:
        stock_price = stock_price_data['Close'].iloc[0]
        print(f"Stock Price on {stock_price_data.index[0].date()}: {stock_price}")

        # Step 4: Calculate the market capitalization
        if total_shares is not None:
            market_cap = stock_price * total_shares
            print(f"Market Cap on {income_statement_date}: {market_cap}")
        else:
            print("Cannot calculate market cap due to missing total shares data.")
    else:
        print(f"No stock price data found around {income_statement_date}")
except Exception as e:
    print(f"Error fetching stock price data: {e}")


Quarterly Income Statement:
                                                       2024-03-31  \
Tax Effect Of Unusual Items                                   0.0   
Tax Rate For Calcs                                          0.158   
Normalized EBITDA                                   30736000000.0   
Net Income From Continuing Operation Net Minori...  23636000000.0   
Reconciled Depreciation                              2836000000.0   
Reconciled Cost Of Revenue                          48482000000.0   
EBITDA                                              30736000000.0   
EBIT                                                27900000000.0   
Net Interest Income                                           NaN   
Interest Expense                                              NaN   
Interest Income                                               NaN   
Normalized Income                                   23636000000.0   
Net Income From Continuing And Discontinued Ope...  23636000000.0   
Total 

In [25]:
from sec_api import FloatApi
import pandas as pd

floatApi = FloatApi('bff4c32faa316e6a316e1d45202f033205fa4c3ee3dcf60d83cf7650d5e4a403')

response_aapl = floatApi.get_float(ticker='AAPL') 

df_aapl = pd.json_normalize(response_aapl['data'])
df_aapl['reportedAt'] = pd.to_datetime(df_aapl['reportedAt'])
df_aapl.drop(columns=['id', 'tickers', 'cik'], inplace=True)

# float.outstandingShares
df2_aapl = df_aapl.explode('float.outstandingShares')['float.outstandingShares'].apply(pd.Series)
df2_aapl = df2_aapl.rename(columns={
    "period": "outstandingShares.period", 
    "shareClass": "outstandingShares.shareClass", 
    "value": "outstandingShares.value"})

# float.publicFloat
df3_aapl = df_aapl.explode('float.publicFloat')['float.publicFloat'].apply(pd.Series)
df3_aapl = df3_aapl.rename(columns={
    "period": "publicFloat.period", 
    "shareClass": "publicFloat.shareClass", 
    "value": "publicFloat.value"})

aapl_float = df_aapl.join([df2_aapl, df3_aapl], how='outer').drop(columns=['float.outstandingShares', 'float.publicFloat', 'sourceFilingAccessionNo'])

print(aapl_float)

                   reportedAt periodOfReport outstandingShares.period  \
0   2024-05-02 18:04:25-04:00     2024-03-30               2024-04-19   
1   2024-02-01 18:03:38-05:00     2023-12-30               2024-01-19   
2   2023-11-02 18:08:27-04:00     2023-09-30               2023-10-20   
3   2023-08-03 18:04:43-04:00     2023-07-01               2023-07-21   
4   2023-05-04 18:03:52-04:00     2023-04-01               2023-04-21   
5   2023-02-02 18:01:30-05:00     2022-12-31               2023-01-20   
6   2022-10-27 18:01:14-04:00     2022-09-24               2022-10-14   
7   2022-07-28 18:06:56-04:00     2022-06-25               2022-07-15   
8   2022-04-28 18:03:58-04:00     2022-03-26               2022-04-15   
9   2022-01-27 18:00:58-05:00     2021-12-25               2022-01-14   
10  2021-10-28 18:04:28-04:00     2021-09-25               2021-10-15   
11  2021-07-27 18:03:42-04:00     2021-06-26               2021-07-16   
12  2021-04-28 18:02:54-04:00     2021-03-27       

C:\Users\Matthew\AppData\Local\Temp\ipykernel_5540\2406854972.py:9: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_aapl['reportedAt'] = pd.to_datetime(df_aapl['reportedAt'])
